In [1]:
import pandas as pd
import json
import requests

In [2]:
def format_date(date):
    """
    Функция принимает на вход дату в формате день/месяц/год и возвращает в формате год-месяц-день
    
    Параметры:
        date: string — дата
    """
    
    day, month, year = date.split("/")
    
    return "{}-{}-{}".format(year, month, day)

In [6]:
def get_exchange_rates(date_start, date_end, save=False, filename=None):
    """
    Функция принимает на вход начало и конец периода, за которые нужно получить данные о курсе обмена
    ангольской квазы на американский доллар.
    Возвращает DataFrame со столбцами [Date, Value]. Если указана опция save, то сохраняет созданный 
    DataFrame с указанным именем в формате .xlsx
    
    Параметры:
        date_start: string — начало периода,
        date_end: string — конец периода,
        save: boolean — сохранять данные в excel-файл или нет, по умолчанию save=False,
        filename: string — если save=True, то является обязательным, название excel-файла,
        web_driver: string — путь к драйверу Firefox
    """
    
    if save and not filename:
        raise ValueError("Filename should be specified when save option is True")
        
    date_start_formatted = format_date(date_start)
    date_end_formatted = format_date(date_end)
    
    # осуществляем запрос к сайту национального банка Хорватии с указанным промежутком дат
    request_string = "https://www.bna.ao/service/rest/taxas/get/evolucao/taxa/intervalo?datainicio={}&datafim={}&tipocambio=M&moeda=USD"
    request_string = request_string.format(date_start_formatted, date_end_formatted)
    request = requests.get(request_string, verify=False)
    
    # переводим полученный JSON в DataFrame
    exchange_rates_df = pd.read_json(request.text[19:-16], orient='records')
    
    # выбираем нужные столбцы из DataFrame
    exchange_rates_df = exchange_rates_df[['data', 'taxa']]
    
    # форматируем DataFrame
    exchange_rates_df = exchange_rates_df.rename(columns={"data": "Date", "taxa": "AOA"})
    exchange_rates_df['Date'] = pd.to_datetime(exchange_rates_df['Date'], format='%Y-%m-%d')
    
    # заполянем пропущенные даты значениями за ближайшие прошлые даты
    date_range = pd.date_range(start=date_start_formatted, end=date_end_formatted)
    exchange_rates_df = exchange_rates_df.set_index('Date')
    exchange_rates_df = exchange_rates_df.reindex(date_range, fill_value=None)
    exchange_rates_df = exchange_rates_df.reset_index(names='Date')
    exchange_rates_df = exchange_rates_df.fillna(method='bfill')
    
    # ещё раз форматируем DataFrame
    exchange_rates_df['Date'] = exchange_rates_df['Date'].dt.strftime('%d/%m/%Y')
    exchange_rates_df['AOA'] = exchange_rates_df['AOA'].astype(float)
    
    if save:
        exchange_rates_df.to_excel(filename, index=False)
        
    return exchange_rates_df

In [7]:
date_start = "10/10/2022"
date_end = "18/10/2022"

In [8]:
get_exchange_rates(date_start, date_end, save=True, filename="exchange_rates_aoa.xlsx")

C:\Users\Anton\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.bna.ao'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


,Date,AOA
0,10/10/2022,440.63000
1,11/10/2022,441.57595
2,12/10/2022,439.58815
3,13/10/2022,440.97215
4,14/10/2022,442.51065
5,15/10/2022,443.60045
6,16/10/2022,443.60045
7,17/10/2022,443.60045
8,18/10/2022,447.89820
